In [1]:
import time
import os

import IPython.display as ipd
import numpy as np
import pandas as pd

import librosa
from pathlib import Path,PureWindowsPath,PurePosixPath
import torchaudio
import torch
import matplotlib.pyplot as plt
import utils
import librosa

SAMPLE_RATE = 44100

In [2]:
tracks = pd.read_csv("./fma_metadata/tracks.csv")
genres = pd.read_csv("./fma_metadata/genres.csv")
tracks.columns=tracks.iloc[0] 
tracks.columns.values[0] = "track_id"
tracks.drop([0,1],inplace=True)
tracks.track_id = tracks.track_id.astype(int)

C:\Users\manue\AppData\Local\Temp\ipykernel_7988\1660309712.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv("./fma_metadata/tracks.csv")


In [3]:
def GetGenres(path,genre = 'genre_top'):
    id_list = []
    genre_list = []
    for direc in list(path.iterdir()):
        if not direc.is_file():
            for file in (list(direc.iterdir())):
                id_track = str(file)[-10:-4]
                id_list.append(id_track)
                genre_list.append(tracks.loc[tracks.track_id == int(id_track),genre].values[0])
    return np.asarray(id_list),np.asarray(genre_list)


In [4]:
path = Path("./fma_small")
id_list, genre_list = GetGenres(path)

In [5]:
def CreateSpectrograms(load_path,save_path, transformation = "MEL"):
    id_list = []
    genre_list = []
    if transformation == 'MEL':
        transform = torchaudio.transforms.MelSpectrogram(SAMPLE_RATE,n_fft=2048,hop_length=512)
    else:
        transform = torchaudio.transforms.Spectrogram(SAMPLE_RATE,n_fft=2048,hop_length=512)
    for direc in list(load_path.iterdir()):
        if not direc.is_file():
            for file in (list(direc.iterdir())):
                id_track = str(file)[-10:-4]
                try:
                    waveform, sample_rate = torchaudio.load(file)
                    if waveform.shape[0] > 1:
                        waveform = (waveform[0] + waveform[1])/2
                    spec = transform(waveform)
                    torch.save(spec, save_path+"/"+id_track+".pt")
                except:
                    pass


In [6]:
save = "./Spectrograms"
CreateSpectrograms(path,save)

In [ ]:
#Prototype, not sure if it works
class CustomSpectrogramDataset(Dataset):
    def __init__(self, spectrogram,genre, transform=None):
        self.x = spectrogram
        self.target = genre
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.x[idx]
        label = self.target.iloc[idx]
        if (self.transform!=None):
            image = self.transform(image)
        return image, label

In [ ]:
#path = Path("./Spectrograms")

def ChargeDataset(path,id_list,genre_list):
    images = []
    labels = []
    for spec in list(path.iterdir()):
        id_track = str(spec)[13:-3]
        labels.append(genre_list[np.argwhere(id_list == id_track)][0][0])
        images.append(torch.load(spec))
    return np.asarray(images),np.asarray(labels)

#spectograms, genres = ChargeDataset(path,id_list,genre_list)    

In [ ]:
#PLOT FUNCTIONS

def plot_spectrogram(spec, title=None, ylabel="freq_bin", aspect="auto", xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(spec), origin="lower", aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)